In [1]:
import json
import chess
import chess.engine
import openai
import numpy as np
from typing import Tuple
from stockfish import Stockfish
OPENAI_API_KEY = ""
STOCKFISH_PATH = "/usr/local/bin/stockfish" 
GOLDEN_DATA_PATH = "data/my_great_predecessors.jsonl"
GENERATOR_MODEL = "gpt-5-mini"
JUDGE_MODEL = "gpt-4o"
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [2]:
example_entry = {"id":"game57523_move1_w","game_idx":57523,"fen":"r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1","move_san":"Bc2","move_uci":"d3c2","annotation":"1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.","tags":["Positional"],"event":"Positional Understanding #1: Sokolov-Brunner, Oakham 1988","annotator":"https://lichess.org/@/UptownExpress","link":"https://lichess.org/study/0UEt92HK/GZPiI2ay","slm_tag":"Positional","slm_score":5}

In [3]:
def generate_candidate_explanation(fen, move):
    system_prompt = "You are a helpful chess tutor. Explain the strategic reason for the given move."
    user_prompt = f"Position FEN: {fen}\nMove: {move}\nExplain this move:"
    
    try:
        response = client.chat.completions.create(
            model=GENERATOR_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating: {str(e)}"

In [4]:
fen = example_entry['fen']
move = example_entry['move_uci']
golden_comment = example_entry['annotation']

In [5]:
target_comment = generate_candidate_explanation(fen, move)
print("FEN and move:", fen, move)
print("\nGolden comment:\n", golden_comment) 
print("\nTarget comment:\n", target_comment)

FEN and move: r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1 d3c2

Golden comment:
 1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.

Target comment:
 This is a simple repositioning move with several strategic goals.

Key ideas behind Bd3–c2:

- Safer square. On d3 the bishop is somewhat exposed to harassment (knight jumps and pawn pushes). c2 is more secure yet keeps the bishop on the same long light-squared diagonal toward Black’s king (c2–d3–e4–f5–g6–h7).

- Keeps pressure on the kingside. From c2 the bishop still eyes g6/h7 and can participate in a future kingside initiative (for example combined with Qd3, Re1, or pawn storms).

- Improves flexibility and redeployment. From c

## Baseline GCC-Eval

In [6]:
## Source: https://github.com/ml-postech/concept-guided-chess-commentary/blob/master/03_gcc_eval.ipynb
engine_cache = {}

def get_engine_evalutaion(fen, target_move):
    
    key = f"{fen} {target_move}"
    if key in engine_cache.keys():
        return engine_cache[key]

    depth = 20

    stockfish_engine.set_fen_position(fen)
    best_moves = stockfish_engine.get_top_moves(2)
    stockfish_engine.set_depth(depth + 1)
    eval1 = stockfish_engine.get_evaluation()

    stockfish_engine.set_depth(depth + 1)
    target_move_san = target_move.split(" ")[0]
    board = chess.Board(fen)
    uci_text = board.copy().push_san(target_move_san).uci()
    stockfish_engine.make_moves_from_current_position([uci_text])
    eval2 = stockfish_engine.get_evaluation()
    move_score = f"Mate in {eval2['value'] * -1}" if eval2['type'] == "mate" else f"{eval2['value'] * -1}cp"
    if move_score == "Mate in 0": 
        move_score = "Checkmate!"
    if "Mate" not in move_score:
        best_reply = stockfish_engine.get_best_move()
        move_score = f"{move_score}, expected reply - {best_reply}"

    best_move1 = board.copy().san(board.copy().push_uci(best_moves[0]['Move']))
    if best_moves[0]['Mate'] is not None:
        best_score1 = f"Mate in {best_moves[0]['Mate'] - 1}"  
        if best_score1 == "Mate in 0": 
            best_score1 = "Checkmate!"
    else:
        if "cp" in move_score:
            diff = best_moves[0]['Centipawn'] - eval2['value'] * -1
            if diff < 20:
                best_score1 = f"similar to actual move"
            elif diff < 50:
                best_score1 = f"better than actual move over 20cp"
            elif diff < 100:
                best_score1 = f"better than actual move over 50cp"
            else:
                best_score1 = f"better than actual move over 100cp, worth a pawn"
        else:
            best_score1 = f"{best_moves[0]['Centipawn']}cp"

    if len(best_moves) == 1:
        eval_str = f"evaluation: only legal move; move - {target_move_san} {move_score}"
        return eval_str

    best_move2 = board.copy().san(board.copy().push_uci(best_moves[1]['Move']))
    if best_moves[1]['Mate'] is not None:
        best_score2 = f"Mate in {best_moves[1]['Mate'] - 1}" 
        if best_score2 == "Mate in 0": 
            best_score2 = "Checkmate!"
    else:
        if "cp" in move_score:
            diff = best_moves[1]['Centipawn'] - eval2['value'] * -1
            if diff < 20:
                best_score2 = f"similar to actual move"
            elif diff < 50:
                best_score2 = f"better than actual move over 20cp"
            elif diff < 100:
                best_score2 = f"better than actual move over 50cp"
            else:
                best_score2 = f"better than actual move over 100cp, worth a pawn"
        else:
            best_score2 = f"{best_moves[1]['Centipawn']}cp"

    eval_str = f"evaluation: actual move - {target_move_san} {move_score}, best move - {best_move1} {best_score1}, second best move - {best_move2} {best_score2}"

    engine_cache[key] = eval_str
    return eval_str


In [7]:
stockfish_engine = Stockfish(path=STOCKFISH_PATH)
engine_eval = get_engine_evalutaion(fen, move)

In [8]:
print(engine_eval)

evaluation: actual move - d3c2 0cp, expected reply - h7h6, best move - h3 better than actual move over 20cp, second best move - Re1 similar to actual move


In [11]:
JUDGE_MODEL = "gpt-4o"

def get_gcc_baseline_score(fen, move, candidate, reference, engine_eval):
    
    system_prompt = """You will be given two comments about a chess move.
    Your task is to rate the comment on one metric.
    Please make sure you read and understand these instructions carefully.
    
    Evaluation Criteria:
    Relevance (1-5) - Relevence of the target comment to important aspects of the chess move. The comment should include only information relevant to the chess move or reasoning for taking or not taking the chess move. An engine evaluation result and a reference comment is given as a hint. 

    Evaluation Steps:
    1. Read the commment and the reference comment carefully.
    2. Read the chess position and move carefully, and find out important aspects based on the reference.
    3. Assess if every expressen of the comment is relevant to the important information about the chess move.
    4. Assign a Relevance score from 1 to 5.
    """

    user_prompt = f"""position:
    {fen}

    move:
    {move}

    target comment:
    {candidate}

    reference comment:
    {reference}

    engine evaluation:
    {engine_eval}

    Score(1-5, score ONLY): """

    response = client.chat.completions.create(
        model=JUDGE_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1,
        logprobs=True,
        top_logprobs=5,
        temperature=0.0
    )

    top_logprobs = response.choices[0].logprobs.content[0].top_logprobs
    
    score_map = {str(i): 0.0 for i in range(1, 6)}
    total_prob = 0.0
    
    for item in top_logprobs:
        token = item.token.strip()
        if token in score_map:
            prob = np.exp(item.logprob)
            score_map[token] = prob
            total_prob += prob
            
    if total_prob == 0: return 1.0
        
    weighted_score = sum(int(k) * (v / total_prob) for k, v in score_map.items())
    
    return weighted_score

In [12]:
score = get_gcc_baseline_score(fen, move, target_comment, golden_comment, engine_eval)
        
print(f"\nMove: {move}")
print(f"Target comment: {target_comment}")
print(f"Baseline Score: {score:.4f}")


Move: d3c2
Target comment: This is a simple repositioning move with several strategic goals.

Key ideas behind Bd3–c2:

- Safer square. On d3 the bishop is somewhat exposed to harassment (knight jumps and pawn pushes). c2 is more secure yet keeps the bishop on the same long light-squared diagonal toward Black’s king (c2–d3–e4–f5–g6–h7).

- Keeps pressure on the kingside. From c2 the bishop still eyes g6/h7 and can participate in a future kingside initiative (for example combined with Qd3, Re1, or pawn storms).

- Improves flexibility and redeployment. From c2 the bishop can later go to b1/a2 to target e6 or to f5/g6 via different routes; it also frees d3 for a knight or for White’s pieces to use the d-file or the e4 square.

- Prophylaxis. The retreat avoids unpleasant tactics or loss of tempo that could arise if Black could harass the bishop on d3 (…Nb4 or …Bxh2 ideas in some lines). On c2 the bishop is less likely to be forced about.

In short: Bdc2 is a purposeful retreat that pres

## Engine features

In [13]:
def get_interpretable_features(fen, move_uci):
    try:
        board = chess.Board(fen)
        move = chess.Move.from_uci(move_uci)
        
        san = board.san(move)
        is_capture = board.is_capture(move)
        
        board.push(move)
        
        is_check = board.is_check()
        
        piece_square = move.to_square
        attacks = list(board.attacks(piece_square))
        controlled_squares = [chess.square_name(s) for s in attacks]
        
        with chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH) as engine:
            info = engine.analyse(board, chess.engine.Limit(depth=15))
            score = info["score"].white().score(mate_score=10000)
            
            # Check optimality
            board.pop()
            best_info = engine.analyse(board, chess.engine.Limit(depth=15))
            best_score = best_info["score"].white().score(mate_score=10000)
            score_diff = score - best_score
            
        return {
            "move_san": san,
            "is_capture": is_capture,
            "gives_check": is_check,
            "squares_controlled": controlled_squares,
            "eval_centipawns": score,
            "is_optimal": score_diff > -30,
            "is_blunder": score_diff < -200
        }
    except Exception as e:
        return {"error": str(e)}

In [14]:
engine_feats = get_interpretable_features(fen, move)
print(engine_feats)

{'move_san': 'Bc2', 'is_capture': False, 'gives_check': False, 'squares_controlled': ['b1', 'd1', 'b3', 'd3', 'a4', 'e4', 'f5', 'g6', 'h7'], 'eval_centipawns': 4, 'is_optimal': True, 'is_blunder': False}


## Extract atomic facts

In [15]:
def extract_golden_atoms(golden_text):
    prompt = f"""
    You are a chess expert, your task is to break down the following chess commentary into a list of Atomic Facts (e.g. strategic goals, tactical threats, or variations), each item must be a distinct verifiable claim.
    Output pure JSON list of strings.
    
    Commentary: "{golden_text}"
    Output Format: {{"atoms": ["fact1", "fact2"]}}
    """
    system_instruction = """
    You are an Expert Chess Benchmark Creator.
    Your goal is to extract a "Grading Rubric" from human expert commentary to evaluate AI models.
    
    CRITICAL RULES:
    1. Extract ONLY facts related to the **strategic logic, tactics, or evaluation** of the current move/position.
    2. IGNORE non-game content:
       - Historical anecdotes ("In the Parisian Café...", "Deschapelles asserted...")
       - Player biographies or names ("Morphy played this...", "La Bourdonnais..."), or tournament/venue details.
       - Raw notation, a list of algebraic moves (e.g., "4... Nxd4 5. Qxd4 e6").
       - Quotes about other games or generic chess philosophy not applied to this specific board state.
    3. If a sentence mixes history and strategy (e.g., "Fischer played h6 to create luft"), extract ONLY the strategy ("h6 creates luft").
    4. If the commentary contains NO relevant strategic analysis of the current board state, return an empty list [].
    
    OUTPUT FORMAT:
        You must return a valid JSON object with a single key "atoms", which is a list of strings.
        Example: {"atoms": ["Nf3 develops the knight", "Nf3 controls d4"]}
    """
    demos = [
            {
                "text": "The move Nf3 is a standard developing move that controls the center.",
                "atoms": [
                    "Nf3 is a developing move.",
                    "Nf3 controls the center."
                ]
            },
            {
                "text": "In the Parisian Café de la Régence they preferred 2 f4. McDonnell several times played this against La Bourdonnais, but the Frenchman was more successful.",
                "atoms": [] 
            },
            {
                "text": "Although Staunton disliked this, 2... Nc6 develops the knight to a natural square and prepares to fight for d4.",
                "atoms": [
                    "2... Nc6 develops the knight.",
                    "The knight goes to a natural square.",
                    "2... Nc6 prepares to fight for d4."
                ]
            }
        ]
    prompt_text = "Task: Extract the strategic atomic facts for the grading rubric.\n\n"
        
    for demo in demos:
        prompt_text += f"Input: \"{demo['text']}\"\n"
        prompt_text += f"Output: {json.dumps(demo['atoms'])}\n\n"
        
    prompt_text += f"Input: \"{golden_text}\"\nOutput:"
    
    response = client.chat.completions.create(
        model=JUDGE_MODEL,
        messages=[{"role": "system", "content": system_instruction},
                  {"role": "user", "content": prompt_text}],
        response_format={"type": "json_object"}
    )
    try:
        data = json.loads(response.choices[0].message.content)
        return data.get("atoms", []) 
    except:
        return [golden_text] 

In [16]:
golden_atoms = extract_golden_atoms(golden_comment)


In [17]:
for atom in golden_atoms: print(atom)

1.Bc2 is an aggressive improvement.
1.Bc2 is a flexible improvement.
1.Bc2 reinforces d4 indirectly by supporting Qd3.
1.Bc2 prepares Bb3 to increase pressure on d5.
1.Bc2 clears d3 for the queen.
The setup Bc2 + Qd3 often creates direct kingside threats.
The setup Bc2 + Qd3 can quickly put Black’s king under pressure.


## Judge process

In [18]:
def get_weighted_score(fen, move, candidate, golden_atoms, engine_features):
    
    critique_prompt = f"""
    [CONTEXT]
    Position: {fen}
    Move: {move}
    Candidate Explanation: "{candidate}"
    
    [RUBRIC]
    Golden Atoms: {json.dumps(golden_atoms)}
    Engine Facts: {json.dumps(engine_features)}
    
    Task: Critique the candidate.
    1. Did it cover the Golden Atoms?
    2. Did it contradict the Engine Facts (Hallucination)?
    Output: A concise critique.
    """
    
    critique_res = client.chat.completions.create(
        model=JUDGE_MODEL,
        messages=[{"role": "user", "content": critique_prompt}]
    )
    reasoning = critique_res.choices[0].message.content

    score_prompt = f"""
    Based on this critique, rate the explanation from 1 to 5.
    Critique: {reasoning}
    Output the score (1-5) ONLY.
    """

    score_res = client.chat.completions.create(
        model=JUDGE_MODEL,
        messages=[{"role": "user", "content": score_prompt}],
        max_tokens=1,
        logprobs=True,
        top_logprobs=5,
        temperature=0.0
    )

    top_logprobs = score_res.choices[0].logprobs.content[0].top_logprobs
    
    score_map = {str(i): 0.0 for i in range(1, 6)}
    total_prob = 0.0
    
    for item in top_logprobs:
        token = item.token.strip()
        if token in score_map:
            prob = np.exp(item.logprob)
            score_map[token] = prob
            total_prob += prob
            
    if total_prob == 0: return 1.0, reasoning
        
    weighted_score = sum(int(k) * (v / total_prob) for k, v in score_map.items())
    
    return weighted_score, reasoning

In [20]:
new_score, reasoning = get_weighted_score(fen, move, target_comment, golden_atoms, engine_feats)
        
print(f"Final Weighted Score: {new_score:.4f}")
print(f"Judge Reasoning: {reasoning}")

Final Weighted Score: 4.0006
Judge Reasoning: The candidate explanation provides a detailed analysis of the move Bc2 and aligns well with several of the Golden Atoms: 

1. It highlights that Bc2 is a flexible improvement by mentioning the various strategic options available to White after the move, such as attacking different squares and maintaining pressure on the kingside.
2. The explanation emphasizes the safety and flexibility of the bishop on c2, indirectly covering the idea that Bc2 reinforces d4 by supporting Qd3, as it notes that the bishop remains active along the diagonal toward Black’s king.
3. It notes that the move clears d3 for White's pieces, indirectly relating to the Golden Atom that Bc2 clears d3 for the queen.

However, the candidate did not explicitly state that Bc2 prepares Bb3 to increase pressure on d5 or that the setup Bc2 + Qd3 can create direct kingside threats and put Black’s king under pressure, although these ideas are somewhat implied.

Regarding the Engin

## Questions and todos
1. the current dataset is high-quality but still noisy, most comments are a mix of strategic insight (great) and deep historical context (noise), we need to extract the atomic facts carefully (currently through prompt engineering, other ideas?)
   
> Good example:
>
> 14. c4 { Apparently the decisive mistake: it was wrong to allow Black’s pawns to become passed, since it does not prove possible to blockade them on the light squares. After 14 Nd2 a4 15 Bc4 a3 16 b3 Bb7 White would simply have stood worse (the opponent has the two bishops, the centre, and so on), but he could still have held on. }

> Noisy example:
>
> 2. Nf3 { In the Parisian Café de la Régence they preferred 2 f4 – in his time Deschapelles had asserted that ‘any other move is advantageous to Black’, and McDonnell several times played this against La Bourdonnais, but after 2 ... Nc6 3 Nf3 e6 4 c3 d5 5 e5 f6 and ... Nh6 the Frenchman was more successful (+4 -8 =1). Later Staunton also thought that 2 Nf3 was a mistake and that 2 f4, as Saint-Amant played against him, was better. But Morphy categorically disagreed with this, calling 2 f4 ‘a completely incorrect method of play’, and the moves 2 Nf3 and 2 d4 ‘the strongest’. It is clear, wouldn’t you agree, which of these disputants could see into the future ... }
>
> 4. Nxd4 e5 { The exclamation mark is for the breakthrough in time! La Bourdonnais makes a move which became the starting point of a variation developed 150 years(!) later by grandmaster Sveshnikov. The 12th game of the match went 4 ... Nxd4? 5 Qxd4 e6 6 Bc4?! (6 Nc3! is more accurate) 6 ... Ne7 7 Nc3 Nc6 8 Qd1 Bc5 9 0-0 0-0 with a slightly inferior game. }


2. How to compare baseline and our score

In [21]:
def compare(entry, target_comment):
    fen = entry['fen']
    move = entry['move_uci']
    golden_comment = entry['annotation']
    print("FEN and move:", fen, move)
    print("\nGolden comment:\n", golden_comment) 
    print("\nTarget comment:\n", target_comment)
    engine_eval = get_engine_evalutaion(fen, move)
    baseline_score = get_gcc_baseline_score(fen, move, target_comment, golden_comment, engine_eval)
    
    engine_feats = get_interpretable_features(fen, move)
    golden_atoms = extract_golden_atoms(golden_comment)
    new_score, reasoning = get_weighted_score(fen, move, target_comment, golden_atoms, engine_feats)
     

    print(f"Baseline Score: {baseline_score:.4f}")
    print(f"Final Eval Score: {new_score:.4f}")
    print(f"Judge Reasoning: {reasoning}")
    

In [22]:
compare(example_entry, "Move the piece to a stronger position, enhancing its board control.")

FEN and move: r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1 d3c2

Golden comment:
 1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.

Target comment:
 Move the piece to a stronger position, enhancing its board control.
Baseline Score: 2.1975
Final Eval Score: 3.0008
Judge Reasoning: The candidate's explanation partially covered the Golden Atoms. The phrase "Move the piece to a stronger position, enhancing its board control" can be seen as aligning with the ideas of "Bc2 is an aggressive, flexible improvement" and "Bc2 reinforces d4 indirectly," though it did not explicitly mention these points. The explanation did not touch upon other key elements such as supporting Qd3, clearing

In [23]:
qwen_max = """The move d3c2 (Nc2 in algebraic notation) is a positional retreat that serves several strategic purposes:
1. Reinforcing the d4-pawn: The knight on d3 currently defends the central d4-pawn. By moving to c2, it may appear to abandon that defense, but this is often part of a regrouping plan-freeing the d3-square for another piece (like the queen or bishop) or preparing to reroute the knight to a more effective outpost (e.g., via b4 or e3).
2. Avoiding exchanges: On d3, the knight is vulnerable to ...Ne4 or ... Nb4 ideas from Black, potentially forcing an unfavorable exchange. Retreating to c2 sidesteps such tactics while maintaining flexibility.
3. Supporting queenside expansion: From c2, the knight eyes the b4 square, which could become a strong outpost if White plays a3-b4 later. This aligns with long-term plans of gaining space on the queenside.
4. Improving coordination: The current position features both bishops and knights somewhat passively placed. Nc2 clears the d3-square for the light-squared bishop or queen to potentially occupy, improving overall piece harmony.
In sum, Nc2 is a prophylactic and flexible maneuver, * riting long-term piece improvement over"""
compare(example_entry, qwen_max)

FEN and move: r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1 d3c2

Golden comment:
 1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.

Target comment:
 The move d3c2 (Nc2 in algebraic notation) is a positional retreat that serves several strategic purposes:
1. Reinforcing the d4-pawn: The knight on d3 currently defends the central d4-pawn. By moving to c2, it may appear to abandon that defense, but this is often part of a regrouping plan-freeing the d3-square for another piece (like the queen or bishop) or preparing to reroute the knight to a more effective outpost (e.g., via b4 or e3).
2. Avoiding exchanges: On d3, the knight is vulnerable to ...Ne4 or ... Nb4 ideas from Black, p

In [24]:
gpt = """Move: 1. Bc2 (d3c2)
Positional idea (long-term):
• The bishop "steps back" but actually keeps the same attacking diagonal toward h7 (c2-d3-e4-f5-g6-h7). This is a common regrouping: you preserve kingside pressure while making the bishop harder to hit with tempo.
• By vacating d3, White also opens the d-file/diagonal for the queen (the queen on d1 now has a clearer relationship to the center), which can help with plans like Re1 + e4 or central play later.
• From c2, the bishop is also better placed to support a central expansion with e4 (it helps control e4 and the kingside squares behind it), fitting a typical "build-up then break" strategy.
Tactical/short-term point:
• The bishop on d3 can often be met by ...Nb4 (from c6), gaining a tempo by attacking it. Playing Bc2 is a practical way to anticipate that idea and keep your structure intact-if Black plays …..Nb4 anyway, White usually has calm retreats (Bb1/Bd3) without losing anything concrete.
In short: Bc2 is a "quiet" retreat that preserves h7 pressure, improves flexibility, and reduces the chance Black gains easy tempi by chasing your bishop.
"""
compare(example_entry, gpt)

FEN and move: r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1 d3c2

Golden comment:
 1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.

Target comment:
 Move: 1. Bc2 (d3c2)
Positional idea (long-term):
• The bishop "steps back" but actually keeps the same attacking diagonal toward h7 (c2-d3-e4-f5-g6-h7). This is a common regrouping: you preserve kingside pressure while making the bishop harder to hit with tempo.
• By vacating d3, White also opens the d-file/diagonal for the queen (the queen on d1 now has a clearer relationship to the center), which can help with plans like Re1 + e4 or central play later.
• From c2, the bishop is also better placed to support a central expansion wit

In [25]:
gemini = """Positional Explanation: Building a Kingside Battery
This is a classic strategic move in symmetrical pawn structures (like the French Exchange).
White's primary intention is to construct a battery on the b1-h7 diagonal.
• Plan: By retreating the bishop to c2, White clears the d3 square for the Queen. The follow-up move is often Qd3, which aligns the Queen and Bishop to aim directly at Black's h7 pawn.
• Provocation: This battery forces Black to react to the checkmate threat, often compelling them to play a move like ...g6 or ...h6. These pawn moves loosen Black's kingside defensive structure, creating "hooks" and weaknesses that White can exploit later in the attack.
• Piece Preservation: The bishop on d3 is a valuable attacking piece. Moving it to c2 keeps it safe from harassment (like a future ...Nb4, though currently prevented by a3) and prevents Black from forcing a trade, ensuring White maintains their attacking potential.
"""
compare(example_entry, gemini)

FEN and move: r1bq1rk1/pp3ppp/2nb1n2/3p4/3P4/P1NB4/1P2NPPP/R1BQ1RK1 w Qq - 0 1 d3c2

Golden comment:
 1.Bc2 is an aggressive, flexible improvement. It reinforces d4 indirectly by supporting Qd3, prepares Bb3 to increase pressure on d5, and—crucially—clears d3 for the queen. The standard setup Bc2 + Qd3 often creates direct kingside threats and can quickly put Black’s king under pressure. Internalize this pattern; it should become automatic.

Target comment:
 Positional Explanation: Building a Kingside Battery
This is a classic strategic move in symmetrical pawn structures (like the French Exchange).
White's primary intention is to construct a battery on the b1-h7 diagonal.
• Plan: By retreating the bishop to c2, White clears the d3 square for the Queen. The follow-up move is often Qd3, which aligns the Queen and Bishop to aim directly at Black's h7 pawn.
• Provocation: This battery forces Black to react to the checkmate threat, often compelling them to play a move like ...g6 or ...h6. 